In [ ]:
import requests
import json
import pickle
import time
import csv

## [abo | gun | blm]
campaign = 'blm'

url_tweet_cascade = 'http://130.56.248.5:9357/active-tweets/_search?pretty&scroll=1m'
url_tweet_cascade_scroll = 'http://130.56.248.5:9357/_search/scroll?pretty'

video_ids = []
with open('../data/social_media/{}/video_annotations.csv'.format(campaign)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        video_id = row[0]
        label = row[3]
        if label == '1':
            video_ids.append(video_id)
        line_count += 1
    print('Processed {} lines.'.format(line_count))
    
print('len(video_ids):', len(video_ids))

num_tweets_per_video = {}
tweets = []
total = 0

for video_id in video_ids:
    scroll_ids = []
    query_json = {
        "size": 5000,
        "query": {
            "bool":{
                "should":[
                    {"match_phrase": {"original_vids": video_id}},
                    {"match_phrase": {"retweeted_vids": video_id}},
                    {"match_phrase": {"quoted_vids": video_id}}
                ],
                "minimum_should_match": 1
            },
        }
    }

    query_str = json.dumps(query_json)

    headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

    r = requests.post(url_tweet_cascade, data=query_str, headers=headers)
    result = json.loads(r.content.decode('utf-8'))
    #print(result)
    hits = result['hits']['hits']
    #print("# returned hits:", len(hits))
    num_total_hits = result['hits']['total']
    print(video_id)
    print("# total hits:", num_total_hits)
    num_tweets_per_video[video_id] = num_total_hits
    print("returned hits: ", len(result['hits']['hits']))
    #print("max_score:", result['hits']['max_score'])
    last_scroll_id = result['_scroll_id']
    scroll_ids.append(last_scroll_id)
    total += len(hits)
    
    for hit in hits:
        tweets.append(hit)
    
    ## Scroll search
    while len(hits) > 0:
        try:
            scroll_query_json = {
                "scroll" : "1m", 
                "scroll_id" : last_scroll_id
            }

            scroll_query_str = json.dumps(scroll_query_json)
            r = requests.post(url_tweet_cascade_scroll, data=scroll_query_str, headers=headers)
            result = json.loads(r.content.decode('utf-8'))
            hits = result['hits']['hits']
            print("returned hits:", len(hits))
            total += len(hits)
            #print("# total:", total)
            last_scroll_id = result['_scroll_id']
            scroll_ids.append(last_scroll_id)
            for hit in hits:
                tweets.append(hit)
        except:
            print(result)
            break
    

    ## Clear/Close the sessions
    clear_scroll_query_json = {"scroll_id": scroll_ids}
    clear_scroll_query_str = json.dumps(clear_scroll_query_json)
    r = requests.delete(url_tweet_cascade_scroll, data=clear_scroll_query_str, headers=headers)

    print("# tweets:", len(tweets))
    print("# total:", total)
    print("=============================================")
    
    time.sleep(0.1)


results = {}
for tweet in tweets:
    tweetId = tweet['_source']['tweet_id_str']
    results[tweetId] = tweet

print("# results: ", len(results.keys()))

try:
    pickle.dump(results, open("../data/social_media/{}/all_tweets_by_relevant_videos.pkl".format(campaign), "wb"))
except:
    print("couldn't dump the tweets!!!")
    pass
